The goal of this notebook is to find parameters for the MOTIS ODM Mixer that satisfy the requirements specified by the respective public transit provider.

In [1]:
# Setup
import os
import subprocess
from violations import violations
from visualize import visualize

motis = "/home/mority/code/motis/cmake-build-release-gcc-14/motis"

In [2]:
# Configure mixer
cfg = \
"""{
    "direct_taxi_penalty": 20.0,
    "pt_slope": 2.2,
    "odm_slope": 2.0,
    "taxi_cost": [[0,20.6],[1,4.9]],
    "transfer_cost": [[0,8.0]]
}"""
with open("mixer.json","w") as f:
    f.write(cfg)
cfg_path = os.path.abspath("mixer.json")

In [3]:
# Mix test input
for cs in os.listdir("in"):
    in_path = os.path.join("in", cs)
    if os.path.isdir(in_path):
        out_path = os.path.join("out", cs)
        os.makedirs(out_path, exist_ok=True)
        subprocess.run(motis + " mixer -c " + cfg_path + " -i " + os.path.join(in_path, "journeys.csv") + " -o " + out_path, shell=True)

In [ ]:
# Find required/forbidden violations and visualize connection sets
for cs in os.listdir("in"):
    in_path = os.path.join("in", cs)
    if os.path.isdir(in_path):
        out_path = os.path.join("out",cs)
        if not os.path.isdir(out_path):
            print("No results for {}".format(cs))
            continue
        if violations(cs):
            visualize(cs)